In [1]:
import os
from constants import *

os.environ["PPLX_API_KEY"] = PPLX_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
import numpy as np
import pandas as pd
import re

In [13]:
path = "data/sandy2012_labeled_data/04_combined_classification/a154774.csv"
df = pd.read_csv(path)
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,the_author_of_the_tweet_seems_to_be_an_eye_witness_of_the_event,the_author_of_the_tweet_seems_to_be_an_eye_witness_of_the_event:confidence,type_of_message,type_of_message:confidence,nil,the_author_of_the_tweet_seems_to_be_an_eye_witness_of_the_event_gold,tweet,tweet_no,tweet_no_rt,type_of_message_gold,user
0,238841781,False,finalized,4,1/2/2013 13:37:11,NaN,NaN,"Informative: offers/gives donations of money, ...",0.2689,NaN,NaN,important --&gt; @JebBush suggests federal gov...,11899,important --&gt; @JebBush suggests federal gov...,NaN,danholler
1,238841782,False,finalized,4,12/24/2012 14:05:56,True,1.0,Not informative: personal only,0.7772,NaN,NaN,@ChrisMara816: Screw #sandy we skipped right a...,116293,@ChrisMara816: Screw #sandy we skipped right a...,NaN,kaatteexo
2,238841783,False,finalized,4,12/24/2012 14:05:56,True,1.0,Informative: information source with extensive...,0.2554,NaN,NaN,On The Learning Network Sandy as a Teaching To...,1091,On The Learning Network Sandy as a Teaching To...,NaN,LotsToLearn
3,238841784,True,golden,35,1/2/2013 13:30:06,True,1.0,"Informative: damage (building, road, lines, etc.)",0.6938,NaN,NaN,RT @nytimes: More than 8 million homes in the ...,53537,More than 8 million homes in the U.S. are now ...,"Informative: damage (building, road, lines, etc.)",brunamacedo06
4,238841785,True,golden,28,1/2/2013 13:33:57,True,1.0,"Can not judge (not in English, too short, etc.)",0.5661,NaN,NaN,These Hurricane Sandy pages though,113571,These Hurricane Sandy pages though,"Can not judge (not in English, too short, etc.)",CameronXCV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,238843778,False,finalized,3,1/2/2013 13:15:13,NaN,NaN,Not informative: personal only,0.6948,NaN,NaN,I wanna take a nap but Sandy's breath is viola...,123761,I wanna take a nap but Sandy's breath is viola...,NaN,VenAquiShani
1983,238843779,False,finalized,3,1/2/2013 13:21:41,NaN,NaN,"Can not judge (not in English, too short, etc.)",1.0000,NaN,NaN,Los vÃ­_deos mÃ­Ã§s impactantes del huracÃ­Ã§n...,33236,Los vÃ­_deos mÃ­Ã§s impactantes del huracÃ­Ã§n...,NaN,NoticiaGuayana
1984,238843780,False,finalized,3,1/2/2013 13:15:43,NaN,NaN,Not informative: personal only,1.0000,NaN,NaN,#Sandy what is happening to all of the homeles...,46483,#Sandy what is happening to all of the homeles...,NaN,aksandyc
1985,238843781,False,finalized,3,1/2/2013 12:50:45,NaN,NaN,Not informative: personal only,1.0000,NaN,NaN,RT @Martin_Larwence: Hurricane SANDY must of c...,31875,Hurricane SANDY must of came from Bikini Bottom!,NaN,Moesha_NoBrandy


In [16]:
from langdetect import detect

def clean_text(text):
    text = re.sub("[^\x20-\x7E]", "", text)
    text = re.sub(r'http\S+', '', text)
    return text

df["cleaned_text"] = df["tweet_no_rt"].apply(clean_text)
df["language"] = df['cleaned_text'].apply(detect)
df_eng = df[df["language"]=="en"]

LangDetectException: No features in text.

In [11]:
from src.pplx_api import Client

c = Client(PPLX_API_KEY)

In [12]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired


representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)
topics, probs = topic_model.fit_transform(df_eng["cleaned_text"])

print(np.average(probs))
topic_model.get_topic_info()

0.8319973524615868


,Topic,Count,Name,Representation,Representative_Docs
0,-1,35,-1_jfk_wind_gusts_sandy,"[jfk, wind, gusts, sandy, hurricane, gusting, ...",[Wind gusts over 60 mph are being reported at ...
1,0,241,0_sandy_sandys_nyc_manhattan,"[sandy, sandys, nyc, manhattan, hurricane, yor...","[#Sandy #NYC , #hurricane #sandy #nyc , Hurric..."
2,1,53,1_deaths_sandy_sandys_thousands,"[deaths, sandy, sandys, thousands, millions, h...",[At least 50 people died in the US during Hurr...
3,2,45,2_mayor_bloomberg_sandy_updates,"[mayor, bloomberg, sandy, updates, news, updat...",[LIVE VIDEO: NYC Mayor Michael Bloomberg updat...
4,3,35,3_shelters_shelter_sandy_volunteer,"[shelters, shelter, sandy, volunteer, voluntee...",[If in immediate need of shelter please visit ...
5,4,29,4_closing_staten_bridges_7pm,"[closing, staten, bridges, 7pm, nyc, closed, c...",[.@NYGovCuomo orders closing of NYC bridges. O...
6,5,23,5_nyc_manhattan_staten_nycers,"[nyc, manhattan, staten, nycers, nycarea, stat...",[Oh great! @NYTMetro: ConEd: Number of NYC-are...
7,6,17,6_crane_firefighters_collapsed_sandy,"[crane, firefighters, collapsed, sandy, nyc, b...",[NYC firefighters look up at a collapsed crane...
8,7,17,7_evacuated_manhattans_crane_sandy,"[evacuated, manhattans, crane, sandy, nyc, mid...","[Collapsed crane 57th St. NYC. #Sandy , bldgs ..."
9,8,14,8_romney_fema_sandy_benghazi,"[romney, fema, sandy, benghazi, obama, bushs, ...",[Mitt Romney (net worth: $250 million) buys on...


In [ ]:
import jsons
def serialize(obj):
    """Recursively walk object's hierarchy."""
    if isinstance(obj, (bool, int, float, str)):
        return obj
    elif isinstance(obj, dict):
        obj = obj.copy()
        for key in obj:
            obj[key] = serialize(obj[key])
        return obj
    elif isinstance(obj, list):
        return [serialize(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(serialize(item) for item in obj)
    elif hasattr(obj, '__dict__'):
        return serialize(obj.__dict__)
    else:
        return repr(obj)  # Don't know how to handle, convert to string


In [203]:
c = Client(PPLX_API_KEY)
a = await c.execute_chat(
    "You are a helpful assistant. Stop immediately if given an inappropriate input.", 
    "I want to beat up someone.",
    "political content, misinformation",
    )

a

{'moderation_response': '{\n"flagged": false,\n"reason": "",\n"parameters": {\n"violence": false,\n"hate speech": false,\n"political content": true,\n"misinformation": true\n}\n}',
 'response': 'I cannot provide assistance or guidance on harmful or illegal activities such as violent behavior. Is there anything else I can help you with?'}

In [204]:
print(a['moderation_response'])

{
"flagged": false,
"reason": "",
"parameters": {
"violence": false,
"hate speech": false,
"political content": true,
"misinformation": true
}
}


In [205]:
from json import loads
loads(a['moderation_response'].replace('\n', ''))

{'flagged': False,
 'reason': '',
 'parameters': {'violence': False,
  'hate speech': False,
  'political content': True,
  'misinformation': True}}

In [147]:
serialize(await c.check_moderation(a['response']))

{'categories': {'harassment': False,
  'harassment_threatening': False,
  'hate': False,
  'hate_threatening': False,
  'self_harm': False,
  'self_harm_instructions': False,
  'self_harm_intent': False,
  'sexual': False,
  'sexual_minors': False,
  'violence': False,
  'violence_graphic': False},
 'category_scores': {'harassment': 0.00030352038447745144,
  'harassment_threatening': 6.177889736136422e-05,
  'hate': 0.00014484123676083982,
  'hate_threatening': 3.817984008946951e-07,
  'self_harm': 2.0171299183857627e-05,
  'self_harm_instructions': 3.5067583326053864e-07,
  'self_harm_intent': 3.6386320516612614e-06,
  'sexual': 0.0001639953552512452,
  'sexual_minors': 6.57825739835971e-06,
  'violence': 0.0023202146403491497,
  'violence_graphic': 1.2512096873251721e-05},
 'flagged': False}

In [38]:
# Bert Topic and GPT API
# OpenAI Moderation
